In [79]:
#Dependencies
from bs4 import BeautifulSoup
import pandas as pd
from splinter import Browser
import datetime as datetime
import time
import json
import re
import numpy as np

In [2]:
#create splinter browser
executable_path = {'executable_path':"../chromedriver.exe"}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:

#define url as variable and open with browser
url = "https://dallasites101.com/archive/2020-dallas-restaurant-openings-closings-and-coming-soons"
browser.visit(url)
time.sleep(1)
html = browser.html
soup = BeautifulSoup(html, 'lxml')

In [4]:
#Grab lists of openings and closings
listings = soup.find_all('ul')

In [5]:
# Save lists of buisness type and status for a nested for loop
list_type = ['Restaurant', 'Restaurant', 'Restaurant', 'Bar', 'Bar', 'Bar', 'Sweets', 'Sweets', 'Sweets', 'Cafe', 'Cafe', 'Cafe']
list_status = ['Open', 'Coming Soon', 'Closed', 'Open', 'Coming Soon', 'Closed', 'Open', 'Coming Soon', 'Closed', 'Open', 'Coming Soon', 'Closed',]


'https://www.duggburger.com'

In [100]:
name =[]
neighborhood = []
kind = []
status = []
hyperlink = []
date = []

#Loop through each ul, then loop through each li.
for x in range(0, (len(listings)-1)):
    for store in listings[x].find_all('li'):
        temp = store.find('p')
        temptext = temp.text.replace("\xa0", " ")
        temptext = temptext.split(' | ')
        name.append(temptext[0])
        temphood = temptext[1].split(' (')[0].split(' \t')[0]
        neighborhood.append(temphood)
        print(temphood)
        kind.append(list_type[x])
        status.append(list_status[x])
        try:
            templink = temp.find('a').get('href')
            hyperlink.append(templink)
        except AttributeError:
            hyperlink.append(np.nan)
        date.append(datetime.datetime.today().strftime("%m/%d/%y"))


Trinity Groves
Oak Cliff
Grapevine
Deep Ellum 
Henderson Ave
Design District
Plano
Bishop Arts
Arlington
Uptown
Deep Ellum
Plano
Downtown Dallas 
Bishop Arts
Legacy Hall
Lower Greenville
Northwest Dallas
East Dallas 
Plano
Preston Hollow
Deep Ellum
Oak Cliff
Henderson 
Greenville Ave
Frisco
West Dallas
Deep Ellum
McKinney
Bluffview
The Hill
Trinity Groves
Lakewood
Trinity Groves
Uptown
East Dallas
Grand Prairie
Downtown Dallas
Plano
Carrollton
Plano
Carrollton
Fitzhugh
Lemmon Ave
Bishop Arts
Deep Ellum
Oak Lawn 
Lakewood
South Dallas
Carrollton
Irving
Lake Highlands
Victory Park
Lower Greenville
Downtown
Lower Greenville
SMU
Lake Highlands 
Lakewood
Preston Royal
Las Colinas
Lakewood
Uptown
Downtown
SMU
Northwest Dallas
Downtown
Delivery
Deep Ellum
Delivery only in Grapevine
Plano
SMU
The Colony
Downtown
Bishop Arts 
Plano
Lower Greenville
West Dallas
Trinity Groves
The Hill
North Dallas
Fair Park
Richardson
Greenville Ave
DeSoto
Downtown
Plano
Downtown
Plano
Henderson
Addison
Plano
Ri

In [101]:
#Load data into dataframe

data = {'Name': name, 'Neighborhood': neighborhood, 'Type': kind, 'Status': status, 'URL': hyperlink, 'Date_Updated': date}

df = pd.DataFrame(data)

In [102]:
df.sample(5)

,Name,Neighborhood,Type,Status,URL,Date_Updated
235,Wah Wah Room,Lower Greenville,Bar,Closed,NaN,12/30/20
190,Harwood Arms,Harwood,Bar,Open,https://harwoodarmsdallas.com,12/30/20
80,Tarantino’s Cicchetti Bar & Record Lounge,Fair Park,Restaurant,Open,https://www.facebook.com/tarantinosrecordlounge/,12/30/20
130,City Cafe,Park Cities,Restaurant,Closed,NaN,12/30/20
21,Encina,Oak Cliff,Restaurant,Open,https://www.encinadallas.com/,12/30/20


In [103]:
#create  unique IDs for SQL tables
df['nameID'] = df.groupby(['Name']).ngroup()
df['neighborhoodID'] = df.groupby(['Neighborhood']).ngroup()
df['typeID'] = df.groupby(['Type']).ngroup()
df['statusID'] = df.groupby(['Status']).ngroup()
df['urlID'] = df.groupby(['URL']).ngroup()


In [105]:
#Change urlID -1 into None
df.loc[df.urlID == -1,['urlID']] = np.nan
df.sample(10)

,Name,Neighborhood,Type,Status,URL,Date_Updated,nameID,neighborhoodID,typeID,statusID,urlID
142,Foxyco,Design District,Restaurant,Closed,NaN,12/30/20,74,15,2,0,NaN
260,Eataly,NorthPark,Cafe,Open,https://www.eataly.com,12/30/20,60,54,1,2,72.0
70,Seely’s Mills,SMU,Restaurant,Open,https://www.thebeemanhotel.com/?utm_campaign=b...,12/30/20,196,70,2,2,134.0
46,Lochland’s Food & Spirits,Lakewood,Restaurant,Open,http://www.lochlandsdallas.com,12/30/20,136,42,2,2,15.0
74,Son of a Butcher,Plano,Restaurant,Open,https://www.legacyfoodhall.com/vendor/son-of-a...,12/30/20,206,62,2,2,105.0
196,Rare Books Bar,Frisco,Bar,Open,https://www.facebook.com/RareBooksBar,12/30/20,180,27,0,2,80.0
184,YaYa Best Tex Mex Yogurt,Bishop Arts,Restaurant,Closed,NaN,12/30/20,267,3,2,0,NaN
97,Zalat,Lemmon Ave,Restaurant,Open,https://zalatpizza.com/,12/30/20,273,46,2,2,148.0
249,The Cookie Society,Frisco,Sweets,Open,https://cookiesociety.com/,12/30/20,232,27,3,2,28.0
93,XOXO Dining Room,East Dallas,Restaurant,Open,http://www.xoxodallas.com,12/30/20,265,21,2,2,17.0


In [ ]:
#Create new tables to load into